<a href="https://colab.research.google.com/github/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023/blob/main/Trabajo_Pr%C3%A1ctico_Algoritmos_Juan_Aroca_Perez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Algoritmos de optimización - Trabajo Práctico<br>
Nombre y Apellidos: Juan Aroca Pérez  <br>
Url: https://github.com/JuanArocaMIAR/03MIAR---Algoritmos-de-Optimizacion---2023.git<br>
Google Colab: https://colab.research.google.com/drive/xxxxxxxxxxxxxxxx <br>
Problema:
>1. Sesiones de doblaje <br>

Descripción del problema: "Se precisa coordinar el doblaje de una película. Los actores del doblaje deben coincidir en las tomas en las que sus personajes aparecen juntos en las diferentes tomas. Los actores de doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de grabación independientemente del número de tomas que se graben. No es posible grabar más de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los servicios de los actores de doblaje sea el menor posible. "







                                        

#Modelo
- ¿Como represento el espacio de soluciones?
- ¿Cual es la función objetivo?
- ¿Como implemento las restricciones?

In [25]:
# Importar librerías necesarias:

import numpy as np
import pandas as pd
from google.colab import files

In [105]:
# Carga del dataset con la información
archivo_subido = files.upload()
archivo_xlsx = list(archivo_subido.keys())[0]
dataframe_actores = pd.read_excel(archivo_xlsx)

Saving Datos problema doblaje(30 tomas, 10 actores).xlsx to Datos problema doblaje(30 tomas, 10 actores) (27).xlsx


In [106]:
# Tratamiento del dataframe para dejarlo óptimo para su uso

# Eliminar filas y columnas vacias provenientes del EXCEL
dataframe_actores = dataframe_actores.dropna(axis=1, how='all')
dataframe_actores = dataframe_actores.dropna(how='all')

# Eliminar la primera fila ya que no aporta nada
dataframe_actores = dataframe_actores.drop(dataframe_actores.index[0])

# Modificar el nombre de las columnas para mayor claridad
columnas_actuales = dataframe_actores.columns
nuevos_nombres = ['Toma'] + ['Actor {}'.format(i) for i in range(1, len(columnas_actuales)-1)] + ['Total']
dataframe_actores.columns = nuevos_nombres

print(dataframe_actores)

     Toma  Actor 1  Actor 2  Actor 3  Actor 4  Actor 5  Actor 6  Actor 7  \
1       1      1.0      1.0      1.0      1.0      1.0      0.0      0.0   
2       2      0.0      0.0      1.0      1.0      1.0      0.0      0.0   
3       3      0.0      1.0      0.0      0.0      1.0      0.0      1.0   
4       4      1.0      1.0      0.0      0.0      0.0      0.0      1.0   
5       5      0.0      1.0      0.0      1.0      0.0      0.0      0.0   
6       6      1.0      1.0      0.0      1.0      1.0      0.0      0.0   
7       7      1.0      1.0      0.0      1.0      1.0      0.0      0.0   
8       8      1.0      1.0      0.0      0.0      0.0      1.0      0.0   
9       9      1.0      1.0      0.0      1.0      0.0      0.0      0.0   
10     10      1.0      1.0      0.0      0.0      0.0      1.0      0.0   
11     11      1.0      1.0      1.0      0.0      1.0      0.0      0.0   
12     12      1.0      1.0      1.0      1.0      0.0      1.0      0.0   
13     13   

In [107]:
# Reseteo de índices para que empiecen en 0
dataframe_actores.reset_index(drop=True, inplace=True)
dataframe_actores.index = range(1, len(dataframe_actores) + 1)

# Mostrar el DataFrame con los nuevos índices
print(dataframe_actores)

     Toma  Actor 1  Actor 2  Actor 3  Actor 4  Actor 5  Actor 6  Actor 7  \
1       1      1.0      1.0      1.0      1.0      1.0      0.0      0.0   
2       2      0.0      0.0      1.0      1.0      1.0      0.0      0.0   
3       3      0.0      1.0      0.0      0.0      1.0      0.0      1.0   
4       4      1.0      1.0      0.0      0.0      0.0      0.0      1.0   
5       5      0.0      1.0      0.0      1.0      0.0      0.0      0.0   
6       6      1.0      1.0      0.0      1.0      1.0      0.0      0.0   
7       7      1.0      1.0      0.0      1.0      1.0      0.0      0.0   
8       8      1.0      1.0      0.0      0.0      0.0      1.0      0.0   
9       9      1.0      1.0      0.0      1.0      0.0      0.0      0.0   
10     10      1.0      1.0      0.0      0.0      0.0      1.0      0.0   
11     11      1.0      1.0      1.0      0.0      1.0      0.0      0.0   
12     12      1.0      1.0      1.0      1.0      0.0      1.0      0.0   
13     13   

In [114]:
def crear_hijos(nodo_actual, dimension):
    hijos = []
    for i in range(dimension):
        if i not in nodo_actual:
            hijos.append({'s': nodo_actual + (i,)})
    return hijos

def cota_inferior(nodo_actual, costes):
    # Convertir las tuplas de índices en una lista de enteros
    indices = list(nodo_actual)
    # Acceder a los datos del DataFrame utilizando loc y los índices convertidos
    return costes.loc[indices, indices].sum().sum()

def ramificacion_y_poda(costes):
    dimension = len(costes)
    mejor_solucion = tuple(i for i in range(dimension))
    cota_superior = cota_inferior(mejor_solucion, costes)

    nodos = []
    nodos.append({'s': (), 'ci': cota_inferior((), costes)})

    iteracion = 0

    while len(nodos) > 0:
        iteracion += 1
        nodo_prometedor = min(nodos, key=lambda x: x['ci'])['s']

        hijos = [{'s': x['s'] + (i,), 'cota_inferior': cota_inferior(x['s'] + (i,), costes)} for i in range(dimension) if i not in nodo_prometedor]

        nodo_final = [x for x in hijos if len(x['s']) == dimension]
        if len(nodo_final) > 0:
            if nodo_final[0]['ci'] < cota_superior:
                cota_superior = nodo_final[0]['cota_inferior']
                mejor_solucion = nodo_final[0]['s']

        hijos = [x for x in hijos if x['ci'] < cota_superior]

        nodos.extend(hijos)
        nodos = [x for x in nodos if x['s'][:-1] != nodo_prometedor]

    print("La solución final es:", mejor_solucion, "en", iteracion, "iteraciones", "para dimension:", dimension)

# Llamar a la función ramificacion_y_poda con el DataFrame
ramificacion_y_poda(dataframe_actores)


KeyError: '[0] not in index'

 La idea para resolver este programa es el empleo del problema tipo visto en la asignatura de **Programación lineal entera** puesto que, para empezar, todas las variables son de tipo entero. Además, cumple con la premisa de que hay que buscar un máximo o mínimo de una función objetivo lineal sujeto a restricciones, que en este caso, dichas restricciones serán que los actores de
doblaje cobran todos la misma cantidad por cada día que deben desplazarse hasta el estudio de
grabación independientemente del número de tomas que se graben y no es posible grabar más
de 6 tomas por día. El objetivo es planificar las sesiones por día de manera que el gasto por los
servicios de los actores de doblaje sea el menor posible.

El objetivo del problema será **minimizar el gasto** total de los actores de doblaje. Por otro lado, como algoritmo de búsqueda de la solución, **vamos a emplear ramificación y poda**, para ello, necesitamos una estructura de árbol. Los actores y las tomas ya están colocados en la tabla.

 Las primeras **preguntas** que debemos hacernos son: ¿Cómo diseñamos el árbol? ¿Qué mecanismo diseñamos para ramificar? ¿Cómo podamos?.

 Una posible manera de **diseñar el árbol** es mediante una lista de tuplas, donde cada tupla representa una asignación parcial. Cada elemento de la tupla corresponde a un actor, y el valor en ese elemento indica a qué toma ha sido asignado ese actor como se vio en teoría. En el código que proporcionaste, una tupla del árbol de soluciones tiene la forma (a1, a2, ..., an), donde a_i es el número de toma a la cual el actor i ha sido asignado.

Tenemos una lista de tuplas que representa nuestra **espacio de soluciones**. Debido al volumen de datos, no es posible considerar una sola tupla como hemos visto en teoría, sería ineficiente. Las asignaciones parciales resultan más eficientes en este caso, donde cada nodo es una asignación parcial, lo que es aconsejable si vamos a realizar la técnica de ramificación y poda.

De este modo, por ejemplo, si tenemos la tupla (1, 3, 0, 2, 4, 5, 7, 6, 8, 9), significa que:

El actor 0 ha sido asignado a la toma 1.
El actor 1 ha sido asignado a la toma 3.
El actor 2 ha sido asignado a la toma 0.
El actor 3 ha sido asignado a la toma 2.
Y así sucesivamente.

**¿Cómo construimos la función objetivo?** Dado que **el objetivo es minimizar el gasto total** por los servicios de los actores de doblaje, necesitamos asignar un costo a cada toma asignada. **La matriz de costos** nos ayudará a determinar este costo para cada asignación de toma por actor.

 Vamos a proveernos de una función para estimar la **cota inferior para una solución parcial**. Tenemos que hacer una estimación, esto es la que nos va a servir para podar. Se trata de una estrategia para conducir al algoritmo a que sea más eficiente, o, a que dirija la búsqueda para encontrar la solución óptima. Para la cota inferior lo que **buscamos es el mínimo de esas columnas** para que nos sirva como función de cota inferior.

#Análisis
- ¿Que complejidad tiene el problema?. Orden de complejidad y Contabilizar el espacio de soluciones

Dado que tenemos 10 actores y 30 tomas, tenemos un espacio de soluciones de 30**10 ya que cada actor puede ser asignado a cualquiera de las 30 tomas. El valor del espacio de soluciones es muy grande por lo que destaca la complejidad de explorar todo el espacio de soluciones y la necesidad de algoritmos como ramificación y poda para encontrar soluciones óptimas de manera eficiente.

El número total de soluciones tiene una expresión factorial. Esto se debe a que cada actor tiene 30 tomas entre las cuales puede ser asignado, y hay 10 actores en total, lo que supone un orden de complejidad O(N!), por lo que la técnica de ramificación y poda puede ser muy útil para reducir el número de búsqueda de soluciones.

#Diseño
- ¿Que técnica utilizo? ¿Por qué?

Lo siguiente será aplicar la **técnica de ramificación y poda**. Hemos definido el conjunto de datos que representa el problema de asignación de los actores por tomas. Hemos definido una matriz de costo asociado a cada actor por cada toma.

A continuación vamos a definir las funciones auxiliares: valor, CI, crear_hijos y ramificación_y_poda. Donde la función **valor** calcula el valor de una solución parcial, **CI** calcula una cota inferior para la solución parcial, **crear_hijos** genera los hijos de un nodo para la asignación de actores por tomas, y, la función principal **ramificación_y_poda** ejecuta el algoritmo ramificación y poda.

En la ramificación y poda se van a generar nodos y ramificar el árbol de búsqueda a través de un bucle. En cada iteración se seleccionará el nodo más prometedor basándose de la cota inferior. Se generarán los jijos mediante la función "crear_hijos". Como es necesario también realizar una poda, se eliminarán los nodos cuya cota inferior es mayor o igual a la mejor solución encontrada hasta el momento.

Ahora lo siguiente sería: **Inicializamos la solución inicialmente con todo vacío**, y, nos vamos a ir guardando una **variable que llamamos NODOS**, los nodos que debemos explorar, los nodos vivos, con lo cual lo que hemos hecho es inicializar la solución.

Realizamos un **bucle iterativo**, lo que hacemos es ramificar, exploramos, **buscamos el nodo prometedor**. Hemos creado una **variable HIJOS** que va a guardar los hijos, porque lo que vamos a hacer posteriormente, es podarlos. **A continuación solamente añadiremos a NODOS a aquellos nodos que hayan pasado el filtro de la poda**.